In [ ]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [ ]:
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'

In [ ]:
df_2018 = getDataframe(2018)

In [ ]:
df_2018 = addTokenizedColumnofTitle(df_2018,None,False)

In [ ]:
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)

In [ ]:
words_2018.head()

In [ ]:
stopWordList = getWordListBasedOnPercent(words_2018,Percent = 25)
print(stopWordList[0],len(stopWordList))

In [ ]:
df_2018 = addTokenizedColumnofTitle(df_2018,stopWordList,False)
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)
words_2018.head()

In [ ]:
df_2018.head()

In [ ]:
priorProbabilities = getPriorProbabilities(df_2018)
words_2018 = obtainDataframeWithClassProbabilities(words_2018, AllClasses, delta, appendClassPrefix)
writeModel(words_2018,'wordlength-model.txt',AllClasses,appendClassPrefix)
words_2018 = renameModelRows(words_2018, AllClasses, appendClassPrefix)
model = [words_2018, priorProbabilities]

In [ ]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData,stopWordList,False)
testData = testData.reset_index()
testData = generateCondClassProb(testData, model)
testData['predicted'] = generatePrediction(testData,AllClasses)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'wordlength-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
print(df_confusion)
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)